# 8.1 Project work 3/30 (Sat)
    Finish up methodology of your analysis
    Any suggestions for further analysis?

# 8.2 Project work 4/1 (Mon)
    Sum up results and prepare slides
    Share summary of your results with teammates

# 8.3 Project work 4/3 (Wed)->Presentation Day

I am thinking about analyzing socio-demographic factors to the number of marijuana dispensaries in California...

1) linear regression--let me see how I can do stats more than just linear.
poverty rate, unemployment rate ----> the number of dispensaries by zipcode
ANOVA the number of dispensaries by zipcode
2) heatmap of poverty rate, unemployment rate
3) mark locations of dispensaries

In [2]:
# Dependencies
from census import Census
from us import states
from config import (census_key, gkey)

import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time

# Census Data Retrieval

In [25]:
# Census API Key
c = Census(census_key, year=2017)
    # for labels, https://api.census.gov/data/2017/acs/acs5/groups.html
census_data = c.acs5.get(("NAME",
                          "B01003_001E", #Population
                          "B01002_001E", #Median Age
                          "B19013_001E", #Household Income
                          "B19301_001E", #Per Capita Income
                          "B17001_002E", #Poverty Count"
                          "B23025_005E"  #Unemployment Count
                         ), 
                         {'for': 'zip code tabulation area:*'}) #California, 'zip code tabulation area:*'

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={"NAME": "Name",
                                      "B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * census_df["Poverty Count"].astype(int) / census_df["Population"].astype(int)

# Add in unemployment Rate (Employment Count / Population)
census_df["Unemployment Rate"] = 100 * census_df["Unemployment Count"].astype(int) / census_df["Population"].astype(int)

# Final DataFrame
census_df = census_df[["Name","Zipcode","Population","Median Age","Household Income",
                       "Per Capita Income","Poverty Rate","Unemployment Rate"]]

# Visualize
print(len(census_df))
census_df.head()

33120


,Name,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Unemployment Rate
0,ZCTA5 84306,84306,345.0,26.9,71806.0,39728.0,0.000000,0.000000
1,ZCTA5 84775,84775,421.0,58.9,-666666666.0,26349.0,6.175772,0.712589
2,ZCTA5 84762,84762,211.0,57.2,-666666666.0,35725.0,0.000000,0.000000
3,ZCTA5 84772,84772,203.0,51.3,33958.0,21016.0,0.000000,0.000000
4,ZCTA5 84781,84781,224.0,31.6,48000.0,21092.0,0.000000,7.142857


In [ ]:
census_df.to_csv("census_Cali.csv", encoding="utf-8", index=False)

## Marijuana Dispensaries

In [7]:
marijuana_df=pd.read_csv('CannabisLicData_cleaned.csv')
marijuana_df.head()

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,CA,Zip Code,County_with leading space,County,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal
0,C10-0000010-LIC,Cannabis - Retailer License,KAREN KISSLER: KAREN KISSLER,"ALTERNATIVES, A HEALTH COLLECTIVE : ALTERNATI...",Corporation,1603 HAMPTON WAY SANTA ROSA,CA,95407,SONOMA,SONOMA,Active,3/19/2019,3/18/2020,N/A for this license type,BOTH
1,C10-0000009-LIC,Cannabis - Retailer License,John Swanston: John Swanston: Sean Eaddy,HORIZON NONPROFIT COLLECTIVE : Email- horizonc...,NaN,1841 EL CAMINO AVE SACRAMENTO,CA,95815,SACRAMENTO,SACRAMENTO,Active,1/31/2019,1/31/2020,N/A for this license type,BOTH
2,C10-18-0000287-TEMP,Cannabis - Retailer Temporary License,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,1632 GATEWAY RD CALEXICO,CA,92231,IMPERIAL,IMPERIAL,Active,12/31/2018,7/29/2019,N/A for this license type,Medicinal
3,C10-18-0000288-TEMP,Cannabis - Retailer Temporary License,Basil Bailey,ORIGINAL BALBOA CAREGIVERS : Email- balboacare...,Corporation,9419 MASON AVE CHATSWORTH,CA,91311,LOS ANGELES,LOS ANGELES,Active,12/31/2018,7/29/2019,N/A for this license type,BOTH
4,C10-18-0000289-TEMP,Cannabis - Retailer Temporary License,Michael O'Leary,"Medallion Wellness, Inc. : Email- michael@meda...",Corporation,341 Business PKWY Atwater,CA,95301,,NaN,Active,12/31/2018,7/29/2019,N/A for this license type,BOTH


In [22]:
print(marijuana_df.columns)
marijuana_zip=marijuana_df['Zip Code'].value_counts()
marijuana_zip.sort_index(inplace=True)
marijuana_zip

Index(['License Number', 'License Type', 'Business Owner',
       'Business Contact Information', 'Business Structure', 'Premise Address',
       'CA', 'Zip Code', 'County_with leading space', 'County', 'Status',
       'Issue Date', 'Expiration Date', 'Activities', 'Adult-Use/Medicinal'],
      dtype='object')


90001    3
90007    1
90008    1
90012    1
90014    1
90015    2
90016    3
90018    1
90019    4
90021    9
90023    4
90024    3
90025    1
90026    1
90027    1
90028    1
90029    3
90032    1
90033    1
90034    1
90036    2
90037    1
90038    3
90039    2
90041    1
90042    1
90045    3
90046    8
90047    2
90048    6
        ..
95542    1
95560    1
95589    1
95616    4
95618    1
95619    1
95620    1
95641    2
95682    4
95713    1
95811    3
95815    7
95816    2
95817    2
95818    2
95820    2
95822    2
95824    1
95826    2
95828    3
95833    1
95838    2
95901    2
95959    1
96001    2
96002    2
96003    1
96019    3
96067    2
96094    2
Name: Zip Code, Length: 263, dtype: int64

## Merge

In [ ]:
pd.merge(census_df,marijuana_zip, how='inner', on='Zip Code')

In [ ]:
merge zip code coordinates with marijuana dataframe